# Introduction to Data Science 2025

# Week 4

In this week's exercise, we look at prompting and zero- and few-shot task settings. Below is a text generation example from https://github.com/TurkuNLP/intro-to-nlp/blob/master/text_generation_pipeline_example.ipynb demonstrating how to load a text generation pipeline with a pre-trained model and generate text with a given prompt. Your task is to load a similar pre-trained generative model and assess whether the model succeeds at a set of tasks in zero-shot, one-shot, and two-shot settings.

**Note: Downloading and running the pre-trained model locally may take some time. Alternatively, you can open and run this notebook on [Google Colab](https://colab.research.google.com/), as assumed in the following example.**

## Text generation example

This is a brief example of how to run text generation with a causal language model and `pipeline`.

Install [transformers](https://huggingface.co/docs/transformers/index) python package. This will be used to load the model and tokenizer and to run generation.

In [ ]:
!pip install --quiet transformers

Import the `AutoTokenizer`, `AutoModelForCausalLM`, and `pipeline` classes. The first two support loading tokenizers and generative models from the [Hugging Face repository](https://huggingface.co/models), and the last wraps a tokenizer and a model for convenience.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

Load a generative model and its tokenizer. You can substitute any other generative model name here (e.g. [other TurkuNLP GPT-3 models](https://huggingface.co/models?sort=downloads&search=turkunlp%2Fgpt3)), but note that Colab may have issues running larger models. 

In [ ]:
MODEL_NAME = 'TurkuNLP/gpt3-finnish-large'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

Instantiate a text generation pipeline using the tokenizer and model.

In [ ]:
pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    device=model.device
)

We can now call the pipeline with a text prompt; it will take care of tokenizing, encoding, generation, and decoding:

In [ ]:
output = pipe('Terve, miten menee?', max_new_tokens=25)

print(output)

Just print the text

In [ ]:
print(output[0]['generated_text'])

We can also call the pipeline with any arguments that the model `generate` function supports. For details on text generation using `transformers`, see e.g. [this tutorial](https://huggingface.co/blog/how-to-generate).

Example with sampling and a high `temperature` parameter to generate more chaotic output:

In [ ]:
output = pipe(
    'Terve, miten menee?',
    do_sample=True,
    temperature=10.0,
    max_new_tokens=25
)

print(output[0]['generated_text'])

## Exercise 1

Your task is to assess whether a generative model succeeds in the following tasks in zero-shot, one-shot, and two-shot settings:

- binary sentiment classification (positive / negative)

- person name recognition

- two-digit addition (e.g. 11 + 22 = 33)

For example, for assessing whether a generative model can name capital cities, we could use the following prompts:

- zero-shot:
	>"""\
	>Identify the capital cities of countries.
	>
	>Question: What is the capital of Finland?\
	>Answer:\
	>"""
- one-shot:
	>"""\
	>Identify the capital cities of countries.
	>
	>Question: What is the capital of Sweden?\
	>Answer: Stockholm
	>
	>Question: What is the capital of Finland?\
	>Answer:\
	>"""
- two-shot:
	>"""\
	>Identify the capital cities of countries.
	>
	>Question: What is the capital of Sweden?\
	>Answer: Stockholm
	>
	>Question: What is the capital of Denmark?\
	>Answer: Copenhagen
	>
	>Question: What is the capital of Finland?\
	>Answer:\
	>"""

You can do the tasks either in English or Finnish and use a generative model of your choice from the Hugging Face models repository, for example the following models:

- English: `gpt2-large`
- Finnish: `TurkuNLP/gpt3-finnish-large`

You can either come up with your own instructions for the tasks or use the following:

- English:
	- binary sentiment classification: "Do the following texts express a positive or negative sentiment?"
	- person name recognition: "List the person names occurring in the following texts."
	- two-digit addition: "This is a first grade math exam."
- Finnish:
	- binary sentiment classification: "Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?"
	- person name recognition: "Listaa seuraavissa teksteissä mainitut henkilönnimet."
	- two-digit addition: "Tämä on ensimmäisen luokan matematiikan koe."

Come up with at least two test cases for each of the three tasks, and come up with your own one- and two-shot examples.

In [ ]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

MODEL_NAME = 'distilgpt2'
device = -1

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    low_cpu_mem_usage=True
)

pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    device=device
)

sentiment_zero_shot = """Do the following texts express a positive or negative sentiment?

Text: This movie was absolutely amazing! I loved every minute of it.
Sentiment:"""

sentiment_one_shot = """Do the following texts express a positive or negative sentiment?

Text: The service was terrible and the food was cold.
Sentiment: negative

Text: This movie was absolutely amazing! I loved every minute of it.
Sentiment:"""

sentiment_two_shot = """Do the following texts express a positive or negative sentiment?

Text: The service was terrible and the food was cold.
Sentiment: negative

Text: I had a wonderful experience at this restaurant.
Sentiment: positive

Text: This movie was absolutely amazing! I loved every minute of it.
Sentiment:"""

print("SENTIMENT CLASSIFICATION - Test Case 1")
print("=" * 50)
print("Zero-shot:")
output = pipe(sentiment_zero_shot, max_new_tokens=5, pad_token_id=tokenizer.eos_token_id)
print(output[0]['generated_text'])
print("\nOne-shot:")
output = pipe(sentiment_one_shot, max_new_tokens=5, pad_token_id=tokenizer.eos_token_id)
print(output[0]['generated_text'])
print("\nTwo-shot:")
output = pipe(sentiment_two_shot, max_new_tokens=5, pad_token_id=tokenizer.eos_token_id)
print(output[0]['generated_text'])

sentiment_zero_shot_2 = """Do the following texts express a positive or negative sentiment?

Text: The product broke after just one day of use. Complete waste of money.
Sentiment:"""

sentiment_one_shot_2 = """Do the following texts express a positive or negative sentiment?

Text: The service was terrible and the food was cold.
Sentiment: negative

Text: The product broke after just one day of use. Complete waste of money.
Sentiment:"""

sentiment_two_shot_2 = """Do the following texts express a positive or negative sentiment?

Text: The service was terrible and the food was cold.
Sentiment: negative

Text: I had a wonderful experience at this restaurant.
Sentiment: positive

Text: The product broke after just one day of use. Complete waste of money.
Sentiment:"""

print("\n\nSENTIMENT CLASSIFICATION - Test Case 2")
print("=" * 50)
print("Zero-shot:")
output = pipe(sentiment_zero_shot_2, max_new_tokens=5, pad_token_id=tokenizer.eos_token_id)
print(output[0]['generated_text'])
print("\nOne-shot:")
output = pipe(sentiment_one_shot_2, max_new_tokens=5, pad_token_id=tokenizer.eos_token_id)
print(output[0]['generated_text'])
print("\nTwo-shot:")
output = pipe(sentiment_two_shot_2, max_new_tokens=5, pad_token_id=tokenizer.eos_token_id)
print(output[0]['generated_text'])

name_zero_shot = """List the person names occurring in the following texts.

Text: John Smith and Mary Johnson attended the conference in New York.
Names:"""

name_one_shot = """List the person names occurring in the following texts.

Text: Alice Cooper met with Bob Dylan at the music festival.
Names: Alice Cooper, Bob Dylan

Text: John Smith and Mary Johnson attended the conference in New York.
Names:"""

name_two_shot = """List the person names occurring in the following texts.

Text: Alice Cooper met with Bob Dylan at the music festival.
Names: Alice Cooper, Bob Dylan

Text: Sarah Williams gave a presentation to Michael Brown.
Names: Sarah Williams, Michael Brown

Text: John Smith and Mary Johnson attended the conference in New York.
Names:"""

print("\n\nPERSON NAME RECOGNITION - Test Case 1")
print("=" * 50)
print("Zero-shot:")
output = pipe(name_zero_shot, max_new_tokens=15, pad_token_id=tokenizer.eos_token_id)
print(output[0]['generated_text'])
print("\nOne-shot:")
output = pipe(name_one_shot, max_new_tokens=15, pad_token_id=tokenizer.eos_token_id)
print(output[0]['generated_text'])
print("\nTwo-shot:")
output = pipe(name_two_shot, max_new_tokens=15, pad_token_id=tokenizer.eos_token_id)
print(output[0]['generated_text'])

name_zero_shot_2 = """List the person names occurring in the following texts.

Text: David Lee and Jennifer Chen discussed the project with Robert Taylor.
Names:"""

name_one_shot_2 = """List the person names occurring in the following texts.

Text: Alice Cooper met with Bob Dylan at the music festival.
Names: Alice Cooper, Bob Dylan

Text: David Lee and Jennifer Chen discussed the project with Robert Taylor.
Names:"""

name_two_shot_2 = """List the person names occurring in the following texts.

Text: Alice Cooper met with Bob Dylan at the music festival.
Names: Alice Cooper, Bob Dylan

Text: Sarah Williams gave a presentation to Michael Brown.
Names: Sarah Williams, Michael Brown

Text: David Lee and Jennifer Chen discussed the project with Robert Taylor.
Names:"""

print("\n\nPERSON NAME RECOGNITION - Test Case 2")
print("=" * 50)
print("Zero-shot:")
output = pipe(name_zero_shot_2, max_new_tokens=20, pad_token_id=tokenizer.eos_token_id)
print(output[0]['generated_text'])
print("\nOne-shot:")
output = pipe(name_one_shot_2, max_new_tokens=20, pad_token_id=tokenizer.eos_token_id)
print(output[0]['generated_text'])
print("\nTwo-shot:")
output = pipe(name_two_shot_2, max_new_tokens=20, pad_token_id=tokenizer.eos_token_id)
print(output[0]['generated_text'])

math_zero_shot = """This is a first grade math exam.

Question: 23 + 45 = ?
Answer:"""

math_one_shot = """This is a first grade math exam.

Question: 12 + 34 = ?
Answer: 46

Question: 23 + 45 = ?
Answer:"""

math_two_shot = """This is a first grade math exam.

Question: 12 + 34 = ?
Answer: 46

Question: 51 + 27 = ?
Answer: 78

Question: 23 + 45 = ?
Answer:"""

print("\n\nTWO-DIGIT ADDITION - Test Case 1")
print("=" * 50)
print("Zero-shot:")
output = pipe(math_zero_shot, max_new_tokens=5, pad_token_id=tokenizer.eos_token_id)
print(output[0]['generated_text'])
print("\nOne-shot:")
output = pipe(math_one_shot, max_new_tokens=5, pad_token_id=tokenizer.eos_token_id)
print(output[0]['generated_text'])
print("\nTwo-shot:")
output = pipe(math_two_shot, max_new_tokens=5, pad_token_id=tokenizer.eos_token_id)
print(output[0]['generated_text'])

math_zero_shot_2 = """This is a first grade math exam.

Question: 36 + 58 = ?
Answer:"""

math_one_shot_2 = """This is a first grade math exam.

Question: 12 + 34 = ?
Answer: 46

Question: 36 + 58 = ?
Answer:"""

math_two_shot_2 = """This is a first grade math exam.

Question: 12 + 34 = ?
Answer: 46

Question: 51 + 27 = ?
Answer: 78

Question: 36 + 58 = ?
Answer:"""

print("\n\nTWO-DIGIT ADDITION - Test Case 2")
print("=" * 50)
print("Zero-shot:")
output = pipe(math_zero_shot_2, max_new_tokens=5, pad_token_id=tokenizer.eos_token_id)
print(output[0]['generated_text'])
print("\nOne-shot:")
output = pipe(math_one_shot_2, max_new_tokens=5, pad_token_id=tokenizer.eos_token_id)
print(output[0]['generated_text'])
print("\nTwo-shot:")
output = pipe(math_two_shot_2, max_new_tokens=5, pad_token_id=tokenizer.eos_token_id)
print(output[0]['generated_text'])

**Submit this exercise by submitting your code and your answers to the above questions as comments on the MOOC platform. You can return this Jupyter notebook (.ipynb) or .py, .R, etc depending on your programming preferences.**